# Описание проекта

В распоряжении имеется выгрузка из базы данных кредитного отдела банка со следующей информацией:
- `children` — количество детей в семье
- `days_employed` — общий трудовой стаж в днях
- `dob_years` — возраст клиента в годах
- `education` — уровень образования клиента
- `education_id` — идентификатор уровня образования
- `family_status` — семейное положение
- `family_status_id` — идентификатор семейного положения
- `gender` — пол клиента
- `income_type` — тип занятости
- `debt` — имел ли задолженность по возврату кредитов
- `total_income` — ежемесячный доход
- `purpose` — цель получения кредита

**Цель исследования:** выяснить, влияют ли на возврат кредита в срок следующие факторы:
- количество детей;
- семейное положение;
- уровень дохода;
- цель кредита.

Полученные выводы будут в дальнейшем использованы при построении модели кредитного скоринга.

Содержание:
1. [Обзор данных](#1)
2. [Заполнение пропусков](#2)
3. [Проверка данных на аномалии и исправления](#3)
4. [Изменение типов данных](#4)
5. [Удаление дубликатов](#5)
6. [Декомпозиция исходного датафрейма](#6)
7. [Категоризация дохода](#7)
8. [Категоризация целей кредита](#8)
9. [Анализ влияния факторов на возврат кредита](#9)
10. [Итоговый вывод](#10)

## Обзор данных
<a id="1"></a>

In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('data.csv')

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод:** 
Датасет состоит из 21525 строк и 12 столбцов. В столбцах `days_employed` и `total_income` есть пропуски.

## Заполнение пропусков
<a id="2"></a>

In [4]:
display(pd.DataFrame(round((data.isna().mean()*100),2), columns=['NaNs, %']).style.format(
    '{:.2f}').background_gradient('coolwarm'))

,"NaNs, %"
children,0.00
days_employed,10.10
dob_years,0.00
education,0.00
education_id,0.00
family_status,0.00
family_status_id,0.00
gender,0.00
income_type,0.00
debt,0.00


In [5]:
# получение максимального и минимального дохода
print('Максимальный ежемесячный доход:', data['total_income'].max())
print('Минимальный ежемесячный доход:', data['total_income'].min())

Максимальный ежемесячный доход: 2265604.028722744
Минимальный ежемесячный доход: 20667.26379327158


**Вывод:**
В столбцах `days_employed` и `total_income` одинаковое число пропущенных значений типа NaN (отсутствующее число).

Возможные причины появления пропусков:
- ошибки при выгрузке данных;
- информация не была представлена заемщиками.

Удаление строк с отсутствующими данными приведет к искажению анализа (т.к. доля пропущенных значений существенна - 10%). Целесообразно заполнить пропуски медианными значениями с учетом категории. Категорией, от которой сильнее всего зависит заработок, является income_type.

In [6]:
income_type_pivot = data.pivot_table(index=['income_type'], values='total_income', aggfunc='median') \
                    .rename(columns={'total_income':'Медианный доход'})
income_type_pivot

,Медианный доход
income_type,
безработный,131339.751676
в декрете,53829.130729
госслужащий,150447.935283
компаньон,172357.950966
пенсионер,118514.486412
предприниматель,499163.144947
сотрудник,142594.396847
студент,98201.625314


In [7]:
def total_income_replace_nans(df):
    
    '''
    Возвращает медианный доход из таблицы income_type_pivot по категории category,
    если в датафрейме df встречается NaN в столбце total_income.
    В ином случае оставляем значение total_income. 
    '''
    
    category = df['income_type']
    if pd.isna(df['total_income']):
        return income_type_pivot.loc[category, 'Медианный доход']
    return df['total_income']

In [8]:
data['total_income'] = data.apply(total_income_replace_nans, axis=1)

# проверка отсутствия нулевых значений в total_income
data['total_income'].isna().sum()

0

## Проверка данных на аномалии и исправления
<a id="3"></a>

In [9]:
data.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,2.152500e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.652253e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,9.804367e+04
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.077982e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.425944e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,1.955499e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


In [10]:
data[['family_status', 'gender', 'income_type', 'purpose']].describe()

,family_status,gender,income_type,purpose
count,21525,21525,21525,21525
unique,5,3,8,38
top,женат / замужем,F,сотрудник,свадьба
freq,12380,14236,11119,797


В столбцах `children` и `days_employed` присутствуют аномалии (отрицательные значения дней). Вероятно, имеет место ошибка в выгрузке, в результате которой положительные значения была изменены на отрицательные. Заменим значения на положительные.

In [11]:
data['children'] = abs(data['children'])
data['days_employed'] = abs(data['days_employed'])

Заметим, что в столбце `days_employed` присутствуют аномально высокие значения (401755).

Возможно, имеет место опечатка либо были введены неверные данные. Учитывая, что для ответов на поставленные вопросы значение стажа не имеет существенного значения, аномальные значения можно обработать следующими способами:
- исключить из анализа столбец `days_employed` путем удаления;
- заменить аномально высокие значения максимально возможным стажем (исходя из возраста, пола, предполагаемого непрерывного трудоустройства с 18 лет);
- заменить медианными значениями.

Для упрощения выберем последний вариант. Предположим, что максимальный возраст заемщика 75 лет, непрерывный стаж с 18 лет. В этом случае максимально возможный стаж в днях составит 20805.

В столбце `days_employed` заменим значения, превышающие 20805, на медианные.

In [16]:
data.loc[data['days_employed'] > 20805, 'days_employed'] = data['days_employed'].median()

Также заменим пропуски в `days_employed` медианными значениями:

In [18]:
data['days_employed'] = data['days_employed'].fillna(data['days_employed'].median())

Далее проверим остальные столбцы на предмет наличия артефактов.

In [19]:
data.query('children == 20').shape[0]

76

У 76 заемщиков 20 детей, что маловероятно. Скорее всего, в данных опечатка (лишний ноль на конце).

In [20]:
# исправляем количество детей '20' на '2'
data.loc[data['children'] == 20, 'children'] = 2
data['children'] = data['children'].astype('int')

In [21]:
data.query('dob_years == 0').shape[0]

101

У 101 клиентов нулевой возраст. Вероятно, возраст заемщика не заносился в базу. Посколько возраст заемщика не имеет существенного значения для ответов на поставленные вопросы, оставим нулевые значения.

In [22]:
data['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

У 1 клиента неопределнный пол 'XNA'. Пол также не влияет на дальнейший анализ, оставим значение.

In [17]:
data['education'].value_counts()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64

В столбце education есть неявные дубликаты (слова в разных регистрах). Исправим на следующих шагах.

## Изменение типов данных
<a id="4"></a>

Изменим вещественный тип данных столбцов `days_employed` и `total_income` на целочисленный:

In [23]:
data['days_employed'] = data['days_employed'].astype('int')
data['total_income'] = data['total_income'].astype('int')

## Удаление дубликатов
<a id="5"></a>

In [24]:
# поиск явных дубликатов
data.duplicated().sum()

54

Датасет содержит 54 полностью повторяющиеся строки. Вероятно, данные по некоторым заемщикам были повторно занесены в базу, можно их удалить.

In [25]:
# удаления явных дубликатов с обновлением индексации
data = data.drop_duplicates().reset_index(drop=True)

Ранее были выявлены неявные дубликаты в столбце `education` (слова в разных регистрах). Приведем значения к одному регистру, используя метод str.lower().

In [26]:
data['education'] = data['education'].str.lower()
data['education'].value_counts()

среднее                15188
высшее                  5251
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

Еще раз проверим датасет на наличие явных дубликатов

In [27]:
data.duplicated().sum()

17

После приведения значений в столбце `education` к одному регистру появилось еще 17 дубликатов, удалим их.

In [28]:
data = data.drop_duplicates().reset_index(drop=True)

## Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма
<a id="6"></a>

Создадим датафрейм, в котором каждому уникальному значению из `education` соответствует уникальное значение `education_id`

In [29]:
education_dict = data[['education_id', 'education']]
education_dict = education_dict.drop_duplicates().reset_index(drop=True)
education_dict

,education_id,education
0,0,высшее
1,1,среднее
2,2,неоконченное высшее
3,3,начальное
4,4,ученая степень


Также создадим датафрейм, в котором каждому уникальному значению из `family_status` соответствует уникальное значение `family_status_id`

In [30]:
family_status_dict = data[['family_status_id', 'family_status']]
family_status_dict = family_status_dict.drop_duplicates().reset_index(drop=True)
family_status_dict

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,Не женат / не замужем


Теперь удалим из исходного датафрейма столбцы `education` и `family_status`, оставив только их идентификаторы: `education_id` и `family_status_id`.

In [31]:
data.drop(columns = ['education', 'family_status'], inplace=True)

## Категоризация дохода
<a id="7"></a>

In [34]:
def income_group(income):
    
    '''
    Возвращает категорию дохода по значению дохода income, используя правила:
    0–30000 — 'E';
    30001–50000 — 'D';
    50001–200000 — 'C';
    200001–1000000 — 'B';
    1000001 и выше — 'A'
    
    '''
    
    if income <= 30000:
        return 'E'
    if income <= 50000:
        return 'D'
    if income <= 200000:
        return 'C'
    if income <= 1000000:
        return 'B'
    return 'A'

In [35]:
# создаем столбец total_income_category с категориями, применяя функцию income_group к столбцу total_income
data['total_income_category'] = data['total_income'].apply(income_group)

## Категоризация целей кредита
<a id="8"></a>

Создадим функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:
- 'операции с автомобилем',
- 'операции с недвижимостью',
- 'проведение свадьбы',
- 'получение образования'.

Выведем список уникальных значений столбца `purpose`.

In [36]:
purposes = data['purpose'].unique()
purposes

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Определим подстроки, по которым можно определить категорию:
- 'авто' соответствует категории 'операции с автомобилем';
- 'недвиж' или 'жил' соответствует категории 'операции с недвижимостью';
- 'свад' соответствует категории 'проведение свадьбы';
- 'образов' соответствует категории 'получение образования'

In [37]:
def purpose_group(purpose):
    
    '''
    Возвращает категорию цели по наличию подстроки в цели purpose, используя правила:
    - если подстрока содержит 'авто', то категория 'операции с автомобилем';
    - если подстрока содержит 'недвиж' или 'жил', то категория 'операции с недвижимостью';
    - если подстрока содержит 'свад', то категория 'проведение свадьбы';
    - в остальных случаях категория 'получение образования'
    '''
    
    if 'авто' in purpose:
        return 'операции с автомобилем'
    if ('недвиж' in purpose) or ('жил' in purpose):
        return 'операции с недвижимостью'
    if 'свад' in purpose:
        return 'проведение свадьбы'
    return 'получение образования'

In [38]:
# создаем новый столбец purpose_category с категориями целей
data['purpose_category'] = data['purpose'].apply(purpose_group)

## Анализ влияния факторов на возврат кредита
<a id="9"></a>

### Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [39]:
def create_pivot(column):
    
    '''Возвращает сводную таблицу по столбцу column'''
    
    pivot = data.pivot_table(index=column, values='debt', aggfunc=['count', 'sum', 'mean'])
    pivot.columns = ['Всего кредитополучателей', 'Количество должников', 'Доля должников']
    
    return pivot

In [40]:
create_pivot('children')

,Всего кредитополучателей,Количество должников,Доля должников
children,,,
0,14091,1063,0.075438
1,4855,445,0.091658
2,2128,202,0.094925
3,330,27,0.081818
4,41,4,0.097561
5,9,0,0.000000


**Вывод 1:**

Существует зависимость между количеством детей и возвратом кредита в срок: наименьшая доля просроченных кредитов у семей без детей детей (7,5%), наибольшая - в семьях с 4 детьями (9,8%).

### Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [41]:
create_pivot('family_status_id').merge(family_status_dict, on='family_status_id', how='left')

,family_status_id,Всего кредитополучателей,Количество должников,Доля должников,family_status
0,0,12339,931,0.075452,женат / замужем
1,1,4151,388,0.093471,гражданский брак
2,2,959,63,0.065693,вдовец / вдова
3,3,1195,85,0.071130,в разводе
4,4,2810,274,0.097509,Не женат / не замужем


**Вывод 2:**

Существует зависимость между семейным положением и возвратом кредита в срок: 
- наиболее склонны к выходам на просрочку заемщику с семейныйм статусом "Не женат / не замужем";
- у вдовцов/вдов наблюдается наименьшая доля просроченных кредитов.

### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Ранее были определены следующие категории доходов:
- 0–30000 — 'E';
- 30001–50000 — 'D';
- 50001–200000 — 'C';
- 200001–1000000 — 'B';
- 1000001 и выше — 'A'

In [42]:
# выводим статистику по категориям дохода
data['total_income_category'].value_counts()

C    16015
B     5042
D      350
A       25
E       22
Name: total_income_category, dtype: int64

Поскольку количество заемщиков в категориях D, A, E незначительное, определим более укрупненные категории. Узнаем 25 и 75 персентиль по столбцу `total_income`

In [43]:
data['total_income'].describe().astype('int')

count      21454
mean      165319
std        98187
min        20667
25%       107623
50%       142594
75%       195820
max      2265604
Name: total_income, dtype: int64

У 25% клиентов доход ниже 107623, у половины доход ниже 145017, у 75% заемщиков - ниже 195813. Исходя из этих данных, в целях более точного прослеживания зависимости между доходом и возвратом кредита, введем новые категории дохода:
- 'низкий' при доходе до 107623;
- 'средний' при доходе от 107624 до 195813;
- 'высокий' при доходе выше 195814

In [44]:
def income_group_new(income):
    
    '''
    Возвращает категорию дохода по значению дохода income, используя правила:
    'низкий' при доходе до 107623;
    'средний' при доходе от 107624 до 195813;
    'высокий' при доходе выше 195814
    
    '''
    
    if income <= 107623:
        return 'низкий'
    if income <= 195813:
        return 'средний'
    return 'высокий'

In [45]:
# создаем новый столбец total_income_category_new c новыми категориями дохода
data['total_income_category_new'] = data['total_income'].apply(income_group_new)

In [47]:
create_pivot('total_income_category_new')

,Всего кредитополучателей,Количество должников,Доля должников
total_income_category_new,,,
высокий,5365,383,0.071389
низкий,5364,427,0.079605
средний,10725,931,0.086807


**Вывод 3:**

Существует зависимость между уровнем дохода и возвратом кредита в срок: 
- у клиентов со средним уровнем дохода (от 107624 до 195813) выходы на просрочку встречаются чаще;
- наименьшая доля просрочки среди клиентов с высоким уровнем дохода.

### Как разные цели кредита влияют на его возврат в срок?

In [48]:
create_pivot('purpose_category')

,Всего кредитополучателей,Количество должников,Доля должников
purpose_category,,,
операции с автомобилем,4306,403,0.093590
операции с недвижимостью,10811,782,0.072334
получение образования,4013,370,0.092200
проведение свадьбы,2324,186,0.080034


**Вывод 4:**

- Наиболее часто возникновение задолженности встречается у заемщиков, которые берут кредит на операции с автомобилем и получение образования;
- Заемщики с кредитом на операции с недвижимостью реже выходят на просрочку.

## Итоговый вывод:
<a id="10"></a>

Такие факторы, как наличие детей, семейное положение, уровень дохода и цель кредита влияют на вероятность возврата кредита в срок. Для банка менее рискованными являются следующие группы клиентов:
- без детей; 
- вдовы/вдовцы; 
- с высоким уровнем дохода;
- целевое использование кредита - операции с недвижимостью.